In [ ]:
import re
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from torch.cuda.amp import GradScaler, autocast


# Preprocesare text
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+", "", text)  # Elimină linkurile
    text = re.sub(r"[^a-zA-Zăâîșțèéêàçûü\s]", "", text)  # Elimină caracterele speciale
    text = re.sub(r"\s+", " ", text).strip()  # Elimină spațiile multiple
    return text


# Dataset personalizat
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }


# Parametri
k_folds = 5
num_epochs = 5
batch_size = 16
learning_rate = 1e-5

# Încărcare și curățare date
data_path = r"SII/train.csv"
train_df = pd.read_csv(data_path)
train_df.dropna(subset=["Text", "Label"], inplace=True)
train_df["Text"] = train_df["Text"].apply(preprocess_text)

# Mapare etichete
label_mapping = {"fake": 0, "biased": 1, "true": 2}
labels = train_df["Label"].map(label_mapping).tolist()
texts = train_df["Text"].tolist()

# Tokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

# K-Fold Cross Validation
kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Pregătire pentru AMP și stocare performanță
scaler = GradScaler()
fold_accuracies = []
best_accuracy = 0
best_model_state = None

for fold, (train_idx, val_idx) in enumerate(kfold.split(texts, labels)):
    print(f"\n----- Fold {fold + 1}/{k_folds} -----")

    train_texts = [texts[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_texts = [texts[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
    val_dataset = NewsDataset(val_texts, val_labels, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4)

    model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=3)
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    loss_fn = CrossEntropyLoss()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs = {key: val.to(device) for key, val in batch.items() if key != "label"}
            label = batch["label"].to(device)

            with autocast():
                outputs = model(**inputs)
                loss = loss_fn(outputs.logits, label)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        model.eval()
        val_preds, val_labels_actual = [], []
        with torch.no_grad():
            for batch in val_loader:
                inputs = {key: val.to(device) for key, val in batch.items() if key != "label"}
                label = batch["label"].to(device)

                outputs = model(**inputs)
                preds = torch.argmax(outputs.logits, axis=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels_actual.extend(label.cpu().numpy())
        accuracy = accuracy_score(val_labels_actual, val_preds)
        print(f"Validation Accuracy: {accuracy:.4f}")

    accuracy = accuracy_score(val_labels_actual, val_preds)
    fold_accuracies.append(accuracy)
    print(f"Fold {fold + 1} Accuracy: {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_state = model.state_dict()

print(f"\nAverage Accuracy across folds: {sum(fold_accuracies) / len(fold_accuracies):.4f}")
print(f"Best Fold Accuracy: {best_accuracy:.4f}")
torch.save(best_model_state, "SII/Tema_SII_model1.pth")
model.load_state_dict(best_model_state)

Mounted at /content/gdrive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-027e8bff71f8>:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code durin

Vectorii de predicții au fost salvați într-un fișier CSV la /content/gdrive/MyDrive/tema_predictie/predictii_model.csv


In [ ]:
full_dataset = NewsDataset(texts, labels, tokenizer)
full_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Crearea modelului pentru antrenare completă
model_full = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=3)
optimizer_full = AdamW(model_full.parameters(), lr=learning_rate, weight_decay=0.01)
loss_fn_full = CrossEntropyLoss()

# Mutare pe dispozitiv
model_full.to(device)

# Antrenare completă
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model_full.train()
    for batch in full_loader:
        optimizer_full.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key != "label"}
        label = batch["label"].to(device)

        with autocast():
            outputs = model_full(**inputs)
            loss = loss_fn_full(outputs.logits, label)

        scaler.scale(loss).backward()
        scaler.step(optimizer_full)
        scaler.update()

# Salvare model antrenat pe întreaga bază de date
torch.save(model_full.state_dict(), "SII/Tema_SII_model_full.pth")